In [39]:
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
from common import cFunction as cf
import numpy as np
import wget
import datetime as dt
# from tabula import read_pdf

In [40]:
dataList = pd.read_csv("https://docs.google.com/spreadsheets/d/1VngqG-m7G8k1587c21MZoheR1Fz3amp1mJtiBvA1Jb0/export?format=csv&gid=0")
print("### The total number of target data is " + str(len(dataList)))

### The total number of target data is 417


In [41]:
dataList = dataList[   dataList['제공항목(데이터셋)'] == '한국감정원 주택거래 현황' ]
dataList

,번호,접속 사이트,제공항목(데이터셋),폴더명,작업자,시작일,종료일,검토자,진행상태,비고,...,파라미터,제공방식.1,데이터타입,참고문서,참고문서타입,주기,비고_파라미터설명,완료여부,API PGM ID,파일명
353,354,공공데이터포털,한국감정원 주택거래 현황,353_한국감정원 주택거래 현황,이신우,NaN,NaN,NaN,NaN,NaN,...,startmonth=시작날&endmonth=끝날&region=지역코드,api,xml,NaN,NaN,NaN,1&201903&30000,"353 중복, 20190419 삭제",TB_OD10353.py,TB_OD10353_yyyymm.csv
354,355,공공데이터포털,한국감정원 주택거래 현황,354_한국감정원 주택거래 현황,NaN,NaN,NaN,NaN,NaN,NaN,...,startmonth=201901&endmonth=201903&region=19000,api,xml,https://www.data.go.kr/comm/file/download.do?a...,NaN,NaN,201901&201903&48000,완료,TB_OD10354.py,TB_OD10354_yyyymm.csv


In [42]:
dataList = dataList.fillna("")
dataList = dataList.reset_index(drop=True)
dataList

,번호,접속 사이트,제공항목(데이터셋),폴더명,작업자,시작일,종료일,검토자,진행상태,비고,...,파라미터,제공방식.1,데이터타입,참고문서,참고문서타입,주기,비고_파라미터설명,완료여부,API PGM ID,파일명
0,354,공공데이터포털,한국감정원 주택거래 현황,353_한국감정원 주택거래 현황,이신우,,,,,,...,startmonth=시작날&endmonth=끝날&region=지역코드,api,xml,,,,1&201903&30000,"353 중복, 20190419 삭제",TB_OD10353.py,TB_OD10353_yyyymm.csv
1,355,공공데이터포털,한국감정원 주택거래 현황,354_한국감정원 주택거래 현황,,,,,,,...,startmonth=201901&endmonth=201903&region=19000,api,xml,https://www.data.go.kr/comm/file/download.do?a...,,,201901&201903&48000,완료,TB_OD10354.py,TB_OD10354_yyyymm.csv


In [46]:
inputUrl = dataList.loc[0, "사이트"]
inputKey = dataList.loc[0, "서비스키"]
inputParameter = dataList.loc[0, "파라미터"]
inputFolder = dataList.loc[0, "폴더명"]
inputFile = dataList.loc[0, "서비스명"]
inputDataType = dataList.loc[0, "데이터타입"]
inputRefUrl = dataList.loc[0, "참고문서"]
inputRefType = dataList.loc[0, "참고문서타입"]
newParameter= dataList.loc[0, "비고_파라미터설명"] ## 비고_파라미터설명 에 입력하면 그 값을 출력하기 위해

In [47]:
def good(inputParameter):    

    aa = (inputParameter.split('&')[0]).split('=')[1]  ## 파라미터에 startmonth=00&endmonth=11&region=22
    bb = (inputParameter.split('&')[1]).split('=')[1]  ## 해당 00 11 22 에 추출할 시작,끝,지역코드를
    cc = (inputParameter.split('&')[2]).split('=')[1]  ## 비고_파라미터설명 에 입력하면
    newParameter                                       ## 그걸 처리해주는 함수
    ffff = (newParameter.split('&'))
    aaaaa = inputParameter.replace(aa,ffff[0]).replace(bb,ffff[1]).replace(cc,"")
    inputParameter=aaaaa
    
    return inputParameter

In [48]:
inputParameter=good(inputParameter)
inputParameter

'startmonth=1&endmonth=201903&region='

In [49]:
url = cf.makeURL(inputUrl,inputKey,inputParameter)
print("fullUrl is " + url)

fullUrl is http://openapi.kab.co.kr/OpenAPI_ToolInstallPackage/service/rest/AptTradingStateSvc/getAptTrdStateCaseOfNbr?&serviceKey=aiaNgZ3r3nPeUj5%2FKuJ10ZNRmyePXavavOu8RZwO%2F84LnOn5KxhWa1%2F5nwxk0Y4M10W0fjCK1SksOQeQsdMu6A%3D%3D&startmonth=1&endmonth=201903&region=


In [50]:
regionCodeData = pd.read_csv("./RegionCode.csv")
regionCode = list(regionCodeData["코드"])


fisrtCount = True

for j in regionCode :
    fullUrl =  url + str(j)
    inputDataType = "xml"
    newDF = pd.DataFrame()
    if (inputDataType == "xml"):
        newDF = cf.xmlProcess(fullUrl)
    elif(inputDataType == "json"):
         newDF = cf.jsonProcess(fullUrl)
    elif(inputDataType == "csv"):
        newDF = cf.csvProcess(fullUrl)
    if (len(newDF) ==0) :
        continue
        
    count = len(newDF.loc[0,"rsRow"].split("|"))

    for i in  range (0, count) :
        newDF[(newDF.loc[0,"rsRow"].split("|")[i]).split(",")[0]] = (newDF.loc[0,"rsRow"].split("|")[i]).split(",")[1]
    del newDF["rsRow"]

    if(fisrtCount == True) :
        mergedata = newDF
        fisrtCount = False
    else :
        mergedata = pd.concat([mergedata, newDF], axis=0)

  regionCd regionNm                                              rsRow
0    11000       서울  200601,7704|200602,12717|200603,17419|200604,1...
  regionCd regionNm                                              rsRow
0    26000       부산  200601,4592|200602,6064|200603,6482|200604,583...
  regionCd regionNm                                              rsRow
0    27000       대구  200601,3684|200602,4797|200603,4925|200604,437...
  regionCd regionNm                                              rsRow
0    28000       인천  200601,2143|200602,3948|200603,4482|200604,413...
  regionCd regionNm                                              rsRow
0    29000       광주  200601,3782|200602,3928|200603,5297|200604,444...
  regionCd regionNm                                              rsRow
0    30000       대전  200601,2112|200602,3072|200603,5838|200604,312...
  regionCd regionNm                                              rsRow
0    31000       울산  200601,1435|200602,2388|200603,2531|200604,247...
  regi

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [53]:
groupKey = ["regionCd", "regionNm"]

newMergedata = pd.melt(mergedata, id_vars= groupKey, var_name = 'YEARMONTH', value_name="volume")

finalData = newMergedata.sort_values(["regionCd", "regionNm"]).reset_index(drop=True)
finalData.head()

,regionCd,regionNm,YEARMONTH,volume
0,11000,서울,200601,7704
1,11000,서울,200602,12717
2,11000,서울,200603,17419
3,11000,서울,200604,14919
4,11000,서울,200605,16277


In [54]:
if(newParameter.split('&')[2] != "") :
    finalData = finalData[finalData.regionCd == newParameter.split('&')[2]].reset_index(drop=True)
finalData.head()

,regionCd,regionNm,YEARMONTH,volume
0,30000,대전,200601,2112
1,30000,대전,200602,3072
2,30000,대전,200603,5838
3,30000,대전,200604,3123
4,30000,대전,200605,2219


In [18]:
mergedata.to_csv("./test2.csv", index=False, encoding="ms949")

In [58]:
mergedata

,200601,200602,200603,200604,200605,200606,200607,200608,200609,200610,...,201808,201809,201810,201811,201812,201901,201902,201903,regionCd,regionNm
0,7704,12717,17419,14919,16277,11574,8649,8463,15880,22346,...,13182,17564,16609,7848,6590,7000,4625,6464,11000,서울
0,4592,6064,6482,5839,6192,4887,3350,3068,5920,5037,...,4420,5715,6672,7358,7549,5724,5600,4900,26000,부산
0,3684,4797,4925,4377,3838,3389,2781,2454,3632,3837,...,5473,6293,5660,5816,4833,6765,4553,5500,27000,대구
0,2143,3948,4482,4134,4216,4732,4043,3191,5485,6528,...,4660,5246,6748,6467,6984,6769,8308,8734,28000,인천
0,3782,3928,5297,4446,3000,3925,2629,2176,3438,3225,...,3403,3943,5138,3435,3368,2605,2582,3271,29000,광주
0,2112,3072,5838,3123,2219,1592,1695,1555,3429,2444,...,3067,4804,3962,3766,3162,4012,1921,2310,30000,대전
0,1435,2388,2531,2479,2927,3248,2630,1907,2392,2902,...,1573,1911,1291,1686,1961,2488,2230,3299,31000,울산
0,17630,20561,28400,25277,27199,23842,18705,17087,31661,37995,...,39819,42811,48660,33136,23726,29746,26341,30477,41000,경기
0,1982,2690,3307,2224,2474,2397,2036,1571,2409,2374,...,4246,2544,3847,2837,3174,3809,2391,2640,42000,강원
0,1383,3003,2964,1804,2051,3648,4178,2540,2743,2258,...,2810,3043,4881,5399,3213,2901,2305,2204,43000,충북


In [59]:
newDF

,regionCd,regionNm,200607,200608,200609,200610,200611,200612,200701,200702,...,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903
0,50000,제주,124,135,277,317,804,388,571,336,...,823,716,501,328,457,455,478,701,516,323


In [60]:
newMergedata

,regionCd,regionNm,YEARMONTH,volume
0,11000,서울,200601,7704
1,26000,부산,200601,4592
2,27000,대구,200601,3684
3,28000,인천,200601,2143
4,29000,광주,200601,3782
5,30000,대전,200601,2112
6,31000,울산,200601,1435
7,41000,경기,200601,17630
8,42000,강원,200601,1982
9,43000,충북,200601,1383
